<font size=8 face="Courier"><b>Final Project Source Code

This is the code use to create the plots in the notebook, `Final_Introduction.ipynb`

# <font color="gray">Import Stuff

In [ ]:
import numpy as np                                                    # Packages for data analysis
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
from scipy.ndimage import gaussian_filter1d
import braingeneers                                                   # Braingeneers code
from braingeneers.analysis.analysis import SpikeData, read_phy_files
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets 

# <font color="blue">Load Data

<font color="blue">loadData</font> loads all the necesary data for analyzing one group. It's input is the number of the groups whose data you wish to analyze. It's "output" are the following global variables containing datasets used by later functions.
* `group_num` - the number of the group whose data we are looking at
* `data_before` - data recorded before stimulation
* `data_stim` - uncurated stimulation pattern
* `data_stim_bad` - curated stimulation pattern, it's "bad" because you can't see the stim patterns
* `data_after` - data recorded right after stimulation

In [ ]:
def loadData(group_number):
    global group_num; global data_before; global data_stim; global data_stim_bad; global data_after 
    
    print(f"loading data for group {str(group_number)}")        # Set global variables used by later functions
    group_num = group_number
    data_before = read_phy_files(f"./data/curated/grp{str(group_num)}_before.zip")
    data_stim = read_phy_files(f"./data/uncurated/grp{str(group_num)}_stim.zip")
    data_stim_bad = read_phy_files(f"./data/curated/grp{str(group_num)}_stim.zip")
    data_after = read_phy_files(f"./data/curated/grp{str(group_num)}_after.zip")
    print("done")

Set some initial `group_num` and run `loadData` so that if the user runs the plotting functions below (without first running `loadData`) it wont automatically produce an error.

In [ ]:
group_num=3
loadData(3)

In [ ]:
#interact_manual( loadData, group_number=(2,6) ) 

# <font color="blue"> Graph Raster Plots

<font color="gray"> plotRaster</font> is a simple helper function. that is used to plot a spike raster for a selected dataset `sd`. Its input is the dataset, `sd`, the intended title of the plot `title` and the matplotlib subplot object `ax`. The `ax` objective is somewhat confusing, but makes sense when you look at it within the context of the `plotRasters` function (below)

In [ ]:
def plotRaster( sd, title, ax):
    idces, times = sd.idces_times()
    ax.scatter(times/1000,idces,marker='|',s=1)   # Creates spike raster
    ax.set_title( title )
    ax.set_xlabel("Time(s)")
    ax.set_ylabel('Unit #')

<font color="blue"> plotRasters </font> plots the spike rasters for `data_before`, `data_after`, `data_stim` and `data_stim_bad`

In [ ]:
def plotRasters():
    print(f"Displaying Spike Rasters for Group {str(group_num)}")
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 10) )
    plotRaster(data_before, "Before Stimulation", axs[0,0] )
    plotRaster(data_after, "After Stimulation", axs[0,1] )
    plotRaster(data_stim, "Uncurated Stimulation", axs[1,0] )
    plotRaster(data_stim_bad, "Curated Stimulation", axs[1,1] )
    plt.show()

In [ ]:
#interact_manual( plotRasters ) 

# <font color="blue"> Compare Stimulation Plots

<font color="gray"> plotStimRaster</font> is a helper function. It plots a spike raster and the activity for the first few `seconds` of a selected dataset `sd`. Its input is the dataset, `sd`, the intended title of the plot `title`, the number of seconds to display, `seconds` and the matplotlib subplot object `ax`. The `ax` objective is somewhat confusing, but makes sense when you look at it within the context of the `plotStimRasters` function (below)

In [ ]:
def plotStimRaster(sd, title, seconds, ax):
    idces, times = sd.idces_times()              # Get coordinates for raster
    pop_rate = sd.binned(bin_size=1)             # Get neural activity (population/firing rate) for everything, in ms
    sigma = 5                                    # smooth this line to make it neater
    pop_rate_smooth = gaussian_filter1d(pop_rate.astype(float),sigma=sigma) 
    t = np.linspace(0,sd.length,pop_rate.shape[0])/1000

    ax.scatter(times/1000,idces,marker='|',s=1) # plot the spike raster
    ax2 = ax.twinx()                           
    ax2.plot(t,pop_rate_smooth,c='r')           # create a second plot on top of the raster of the activity

    ax.set_xlim(0,seconds)                      # plot only first few seconds
    ax.set_title( title )                       # add title and axises
    ax.set_xlabel("Time(s)")
    ax.set_ylabel('Unit #')
    ax2.set_ylabel('Firing Rate')

<font color="blue"> plotStimRasters </font> plots the first few `seconds` of the spike raster and activity (firing rate) for `data_stim` and `data_stim_bad`

In [ ]:
def plotStimRasters(seconds):
    print(f"Displaying Spike Rasters for Group {str(group_num)}")
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5) )
    plotStimRaster(data_stim, "Uncurated Stimulation", seconds, axs[0] )
    plotStimRaster(data_stim_bad, "Curated Stimulation", seconds, axs[1] )
    plt.show()

In [ ]:
#interact_manual( plotStimRasters, seconds=(1,30) ) 

# <font color="blue"> View Electrode Grid

<font color="blue"> neuronLayout </font> plots the 2D locations of the electrods and neural units (neurons) on the MaxWell MEA. The <font color="blue"> electrodes are blue </font> and the <font color="red">neurons are red </font>. The plot is created for a specific group `group_num` (passed as global variable) and a specific dataset, from input `data_name`. 

In [8]:
def neuronLayout(data_name):
    
    # Stim electrodes for each group, and electrodes x/y locations
    grp_stim_electrodes = {2:[15569], 3:[15569, 13810], 4:[15569, 13810], 5:[15569, 6705], 6:[15569, 13810, 19971]}
    stim_xy = { 19971:[2992.5,1575.0], 13810:[2975.0,1085.0], 15569:[2957.5,1225.0], 6705:[1837.5,525.0] }

    # put datasets used by specific team in a dictionary object
    sds = {"data_before":data_before, "data_stim":data_stim, "data_after":data_after}    # 3 datasets data_name is allowed to be
    sd = sds[data_name]                                                                  
    
    # Add background image
    plt.figure(figsize=(15,10))
    img = plt.imread("data/Organoids_Closeup.jpeg")
    plt.imshow(img,  extent=[0, 3850, 0, 2100] ) #, #interpolation='nearest', aspect='auto')
    
    # Plot electrodes
    print(f"Displaying Neuron/Electrode Layout for Dataset, {data_name}, from Group {str(group_num)}")
    electrode_mapping = pd.DataFrame.from_dict( sd.metadata[0], orient="index", columns=['x','y']  ) 
    x = electrode_mapping.x.values                                                       # create electrode mapping from saved metadata
    y = electrode_mapping.y.values
    plt.scatter(x,y,s=4, c='darkorange')                                                                 # plot all electrodes
    
    # Plot neurons
    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():                                            # Get X/Y locations of neurons
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    plt.scatter(neuron_x,neuron_y, c='magenta', s=100 )                                                # plot all neurons
    
    # plot stim electrodes
    stim_x = []
    stim_y = []
    for electrode in grp_stim_electrodes[ group_num ]:
        stim_x.append( stim_xy[electrode][0] )
        stim_y.append( stim_xy[electrode][1] ) 
    plt.scatter(stim_x,stim_y, c='turquoise', alpha=.5, s=400 ) 
    
    #add legend, axises limits, labels,  and title
    legend_elements = [Patch("darkorange","darkorange"), Patch("magenta","magenta"), Patch("turquoise","turquoise") ]   # Create colors in legend
    plt.legend(legend_elements, ["Electrode","Neuron", "Stimulation"])       # Add legend
    plt.xlim(0, 3850)
    plt.ylim(0, 2100)
    plt.xlabel('um')                                                                     # add axises and title
    plt.ylabel('um')
    plt.title(f"Neuron & Electrode Layout for {data_name}")
    plt.show()    

In [ ]:
#interact_manual( neuronLayout, data_name=["data_before","data_stim","data_after"] ) 